In [ ]:
import copy
import itertools

# Initial valid 2x2 sudoku solution template for generating permutations
base_solution = [
    [1, 2, 3, 4],
    [3, 4, 1, 2],
    [2, 1, 4, 3],
    [4, 3, 2, 1]
]

def is_valid_sudoku(grid):
    """Check if a 4x4 grid is a valid 2x2 Sudoku solution."""
    # Check rows and columns
    for i in range(4):
        if len(set(grid[i])) != 4 or len(set(row[i] for row in grid)) != 4:
            return False
    
    # Check 2x2 subgrids
    for start_row in (0, 2):
        for start_col in (0, 2):
            block = set(
                grid[start_row][start_col:start_col+2] +
                grid[start_row+1][start_col:start_col+2]
            )
            if len(block) != 4:
                return False
    return True

def generate_permutations(base_solution):
    """Generate all unique permutations of a valid 4x4 grid by row and column swapping."""
    row_perms = list(itertools.permutations([0, 1, 2, 3]))
    col_perms = list(itertools.permutations([0, 1, 2, 3]))
    unique_solutions = set()
    
    for row_perm in row_perms:
        for col_perm in col_perms:
            # Permute rows and columns based on the current permutation
            permuted_grid = [
                [base_solution[row_perm[r]][col_perm[c]] for c in range(4)]
                for r in range(4)
            ]
            # Check if the grid is valid and add it to unique solutions if it is
            if is_valid_sudoku(permuted_grid):
                unique_solutions.add(tuple(map(tuple, permuted_grid)))
                
    return [list(map(list, grid)) for grid in unique_solutions]

def generate_all_puzzles(solutions):
    """Generate all possible puzzles by removing elements from each solution."""
    all_puzzles = set()
    for solution in solutions:
        # Generate puzzles with different numbers of clues
        for num_clues in range(5, 6):  # Minimum clues needed for a unique solution in a 2x2 grid is typically 6
            indices = list(itertools.product(range(4), repeat=2))
            for clues in itertools.combinations(indices, num_clues):
                puzzle = copy.deepcopy(solution)
                for i in range(4):
                    for j in range(4):
                        if (i, j) not in clues:
                            puzzle[i][j] = 0
                all_puzzles.add(tuple(map(tuple, puzzle)))
    return [list(map(list, puzzle)) for puzzle in all_puzzles]




In [3]:
import hashlib

def calculate_checksum(file_path, algorithm="md5"):
    """Calculate the checksum of a file using the specified algorithm."""
    hash_func = None
    if algorithm == "md5":
        hash_func = hashlib.md5()
    elif algorithm == "sha1":
        hash_func = hashlib.sha1()
    elif algorithm == "sha256":
        hash_func = hashlib.sha256()
    else:
        raise ValueError("Unsupported algorithm. Choose 'md5', 'sha1', or 'sha256'.")

    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_func.update(chunk)

    return hash_func.hexdigest()

In [8]:
# Generate all unique solutions
solutions = generate_permutations(base_solution)
print("All solutions generated")

# Generate all possible puzzles from the solutions
all_puzzles = generate_all_puzzles(solutions)
print("All puzzles generated")

# Display the number of unique puzzles and a few examples
print(f"Total unique puzzles generated: {len(all_puzzles)}\n")

All solutions generated
All puzzles generated
Total unique puzzles generated: 381696



In [11]:
# Open a file to write the puzzles
with open("puzzles.h", "w") as f:
    f.write("#ifndef SUDOKU_PUZZLES_H\n")
    f.write("#define SUDOKU_PUZZLES_H\n\n")
    f.write("#define PUZZLE_COUNT {}\n".format(len(all_puzzles)))
    f.write("#define PUZZLE_SIZE 16\n\n")
    f.write("const char puzzles[PUZZLE_COUNT][PUZZLE_SIZE] = {\n")

    # Write each puzzle in C array format
    for i, puzzle in enumerate(all_puzzles):
        f.write("    {")
        
        for j, row in enumerate(puzzle):
            for k, cell in enumerate(row):
                # Print each cell, replacing 0 with the actual 0, and add commas appropriately
                f.write(str(cell))
                if j != 3 or k != 3:  # Add a comma if it's not the last element
                    f.write(", ")

        f.write("}")
        if i < len(all_puzzles) - 1:
            f.write(",\n")
        else:
            f.write("\n")
    
    f.write("};\n\n")
    f.write("#endif // SUDOKU_PUZZLES_H\n")

print("Puzzles have been written to puzzles.h")
print("checksum: ", calculate_checksum("puzzles.h", "md5"))


Puzzles have been written to puzzles.h
checksum:  cdaeeaf5a26a8770f18757f40305f51b


In [12]:
def parse_puzzle_and_result(line):
    """Parse a line from the simulation output and return puzzle, C result, and assembly result as 4x4 grids."""
    parts = line.split()
    puzzle = parts[2]
    c_result = parts[5]
    asm_result = parts[8]

    # Convert hexadecimal strings into 4x4 grids
    def hex_to_grid(hex_str):
        return [[int(hex_str[i], 16) for i in range(j, j + 4)] for j in range(0, 16, 4)]
    
    return hex_to_grid(puzzle), hex_to_grid(c_result), hex_to_grid(asm_result)

def is_valid_sudoku(grid):
    """Check if a 4x4 grid is a valid 2x2 Sudoku solution."""
    # Check rows and columns
    for i in range(4):
        row = set(grid[i])
        col = set(row[i] for row in grid)
        if len(row) != 4 or len(col) != 4:
            return False
    
    # Check 2x2 subgrids
    for start_row in (0, 2):
        for start_col in (0, 2):
            block = set(
                grid[start_row][start_col:start_col+2] +
                grid[start_row+1][start_col:start_col+2]
            )
            if len(block) != 4:
                return False
    return True

def check_results(puzzle_str, c_result, asm_result):
    """Verify that C and assembly results are valid and consistent with each other."""
    if not is_valid_sudoku(c_result):
        print(f"FAIL: C result is not a valid Sudoku for puzzle {puzzle_str}")
        return False
    if not is_valid_sudoku(asm_result):
        print(f"FAIL: Assembly result is not a valid Sudoku for puzzle {puzzle_str}")
        return False
    if c_result != asm_result:
        print(f"FAIL: C and Assembly results differ for puzzle {puzzle_str}")
        return False
    print(f"PASS: Puzzle {puzzle_str} is correct and consistent")
    return True

# Main code to check each line of puzzle results
simulation_output = """
puzzle : 2010000400420000 c result : 2413132431424231 assembly result : 2413132431424231 PASS
puzzle : 2001000002003004 c result : 2341143242133124 assembly result : 2341143242133124 PASS
puzzle : 0421020003000000 c result : 3421123443122143 assembly result : 3421123443122143 PASS
puzzle : 0014400210000000 c result : 2314413212433421 assembly result : 2314413212433421 PASS
puzzle : 0201400000000014 c result : 3241412314322314 assembly result : 3241412314322314 PASS
puzzle : 0020300020000032 c result : 1423321423414132 assembly result : 1423321423414132 PASS
puzzle : 0102031010000000 c result : 4132231412433421 assembly result : 4132231412433421 PASS
puzzle : 1000030140000010 c result : 1423234141323214 assembly result : 1423234141323214 PASS
"""

# Split the output into lines and check each puzzle
err_cnt = 0
for line in simulation_output.strip().split('\n'):
    puzzle, c_result, asm_result = parse_puzzle_and_result(line)
    flag = check_results(line.split()[2], c_result, asm_result)
    err_cnt = err_cnt+1 if flag else err_cnt

print("Error count: ", err_cnt)


ValueError: invalid literal for int() with base 16: ':'